# Open source data ingestion for RAGs with dlt

## What is data ingestion

The porecess of extracting data form a producer, transporting it to a convenient environment and preparing it for usage by normalizing it, sometimes cleaning and adding metadata.

Sometimes the format in which it appears is structured, and with an explicit schema (e.g. Parquet or a db table)

Most of the time, the format is wakly typed and without explicit schema (e.g. csv and json), in which case some normalization and cleaning is required.


## Why not justuse some python scripts?

* Less work, nomore breaking python scripts
* Make sure your data is of good quality
* Keep your data up to date without having to reload the entire dataset
* Make your data pipelines production ready

## Addition considerations for RAGs

* Pre-processing:
    * Extracting the data form PDFs, json, etc.
    * Making sure the resulting text data is clean

* Chunking: Making sure the text is in manageable segments

## dlt (data load tool)

dlt is a Python library that automates data loading with features like schema creation, normalization and integration adaptability

## How doe sit address our challenges?

* Messy python scripts: with dlt you write minimal python doce, it handles most complexities automatically
* Extracting the data: dlt automatically unnest jsons, types it and figures out the schema
* Data versioning: Each load has an in, is verioned and could be rolled back
* Data quality: You can define "data contracts" that reject data that isn't of the right type
* Scaling:
    * Incremental loading - only load new or changed data
    * Performance management



## Challenges with vector DBs

* Most vector DBs only store embeddings and their metadata
* Extra infrastructur and maintenance costs



## Introducing LanceDB: a scalable open source vector DB

* Stores your data (including text and images), the embeddings and metadata.
* Highly scalableand fast

## Load teh data

We'll first load the data just into LanceDB, without embedding it. LanveDB stores both the data and the embeddings, and can also embed data and queries on the fly.

Somedefinitions:

* A dlt source is a grouping of resources (e.g all your data from Hubspot)
* A dlt resource is a function that yields data (e.g. a function yielding all your Hubspot companies)
* A dlt pipeline is how you ingest your data

Loading the data consists of a few steps:

1. Use the requests library to get the data
2. Define a dlt resource that yields the individual documents
3. Create a dlt pipeline and run it

In [3]:
import requests
import dlt

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
qa_dataset = docs_response.json()

@dlt.resource
def qa_documents():
    for course in qa_dataset:
        yield course['documents']

pipeline = dlt.pipeline(
    pipeline_name="from_json", destination="lancedb", dataset_name="qanda"
)

load_info = pipeline.run(qa_documents, table_name="docuements")

print(load_info)

Pipeline from_json load step completed in 0.22 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x0000022B64C87B20> location to store data
Load package 1721688228.8193974 is LOADED and contains no failed jobs


In [5]:
import lancedb

db = lancedb.connect("./.lancedb/")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___docuements']


In [7]:
db_table = db.open_table('qanda___docuements')

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,ad19c050-006d-54ab-bed2-66d3aef1edb3,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721671783.8362322,jVfpGKVDRSSQzQ
1,e0444f5f-573f-5fab-a556-b72524323b40,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721671783.8362322,AptFLPZ8mv0zAw
2,ffd85169-2f5e-5bc0-9a36-8168b3bbe658,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721671783.8362322,6w9DaYv1qUjkfw
3,d32f47cf-fb2e-55cc-a1f3-b6d0d7a30bf5,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721671783.8362322,/rPqEZyAd6bAgQ
4,3c26c7cb-f0c2-5fc1-b6c3-7e8389b45412,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721671783.8362322,61Y1mjRbuoC66A
...,...,...,...,...,...,...
943,13c7ef59-421e-5d2c-b8af-b1ff08b7ff50,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721671783.8362322,7q5XmdXsz1MZQA
944,6a5e3836-38ee-57b1-a2e9-8dfefc434c80,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721671783.8362322,OiU7xyT6UcDUlQ
945,cc8bf787-1ce9-54bc-ac26-13a909e4c763,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721671783.8362322,6JBNh73u1TiDCA
946,d756443c-f943-5cbf-a0e8-0ea7719d20d5,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721671783.8362322,XAOmTIEcF78tEA


## Load and embed the data

Now we load the same data again(into a new table), but embed it directly with the lancedb_adapter. This consists of the following steps:

1. Define the embedding model to use via ENV variables
2. Define a new pipeline to load the same data and embed the "text" and "question" columns with the lancedb_adapter

You can use any embedding model, form open source to OpenAI. We´ve chosen the all-MinLLM-L6-v2 sentence transformer for speed and simplicity.

In [4]:
import os
from dotenv import load_dotenv
from dlt.destinations.adapters import lancedb_adapter

load_dotenv(override=True)

print(os.getenv('DESTINATION__LANCEDB__EMBEDDING_MODEL'))

pipeline = dlt.pipeline(
    pipeline_name="from_json_embedded",destination="lancedb", dataset_name="qanda_embedded"
)

load_info = pipeline.run(
    lancedb_adapter(qa_documents, embed=["text", "question"]),
    table_name="documents"
)
print(load_info)

all-MiniLM-L6-v2


c:\Users\User\Documents\llm-zoomcamp-dbeta95\llm-zoomcampt-3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Pipeline from_json_embedded load step completed in 12.15 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda_embedded
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x0000022B64095000> location to store data
Load package 1721688231.1588693 is LOADED and contains no failed jobs


In [10]:

print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___docuements', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [12]:
db_table = db.open_table('qanda_embedded___documents')

db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,3e288511-782e-5c54-ad3d-763f2c5fde4d,"[-0.0003509373, -0.06201426, -0.037999906, 0.0...",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721675036.142532,/Ng7t4wWjfGDwA
1,f22d4ff0-e8b1-578c-bd48-f67224a1ce3e,"[0.020011412, -0.011535535, 0.013017191, -0.00...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721675036.142532,8XdgqnlC9z+dMg
2,8d4ae14e-9fb4-5007-8c83-b67ec1c019a3,"[0.014857557, -0.06664991, -0.013571233, 0.023...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721675036.142532,ehMcg+O3Rkz2Ww
3,2b01c32a-8963-54d5-a120-7cafdc16bf52,"[-0.023312038, -0.09461491, 0.056361593, -0.00...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721675036.142532,T9mHXEmVMElJBg
4,55dd8935-427d-51a8-a3e9-c8f4d9966560,"[0.026537651, -0.017796684, 0.0021156122, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721675036.142532,rGPs+X44mZArxw
...,...,...,...,...,...,...,...
943,ac535b7e-2dcc-5cde-adb5-4b691d0f2c98,"[0.016619325, -0.03360315, -0.09334715, -0.021...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721675036.142532,wmf8T1CaQR3PQQ
944,e5c592a1-ef76-5cce-bcdd-0bed21a92b8a,"[0.026872838, -0.0019948885, 0.008369074, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721675036.142532,0/EM7VVPwH2A9g
945,59d8e9db-98ad-5d81-ae34-d122e585117e,"[0.03513754, 0.056265566, 0.024428504, -0.0651...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721675036.142532,SL0cx04C3w16IQ
946,744ca1db-43c7-58b4-a93b-f4fe9965ce54,"[0.033809815, -0.0031219649, 0.0017484472, 0.0...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721675036.142532,3PZhMRYp9rq1Lw


## Create an up to date RAG with dlt and LanceDB

In this demo, we will create a LLM chat bot that has the latest knowledge of the employee handbook of a finctional company. We will be able to chat to it about specific policies like PTO, work from gome etc.

To build this we need to od three hings.

1. The company policies exist i an Notion Page. We will need first to extract the text from these pages.
2. Once extracted, we will want to embed them into vectors and then store them in a vector datacase.
3. This will allow us to create our RAG: a function that would accept a user question, mach it to the informatio stored in the vector database, and 

then send the question + relevant information as input to the LLM.

we will be using the following OSS tools for this:

1. dlt for data ingestion:

    * dlt can easily connecto to any REST_API source (like Notion)
    * it also has integrations with vector databases, like LanceDB
    * I also allows to easily plug in functionality like incremental loading

2. lanceDB as a vector databse:
    * LanceDB is an open source vector database that is very easy to use and integrate into python workflows
    * It is in-process an serverless (like DuckDB), which makes querying and retrieval veru effcient

3. Ollama for RAG:
    * Ollama is open source and allows you to easily tun LLMs locally

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Create a dlt project with rest_api source and lancedb destination

We now create a dlt project using the command `dlt init <source> <destination>`.

This downloads all the modules required for the dlt source (rest api, in this case) into the local directory. See the side panel for the directory structure created.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [6]:
%%cmd
dlt init rest_api lancedb 
y

Microsoft Windows [Versi�n 10.0.19045.4651]
(c) Microsoft Corporation. Todos los derechos reservados.

(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>dlt init rest_api lancedb 


Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in .\.dlt\secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information

(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>

### 2. Add API credentials

To access APIs, databases, or any third-party applications, one might need to specify relevant credentials.

With dlt, we can do it in two ways:

Pass the credentials and any other sensitive information inside .dlt/secrets.toml

In [ ]:
[sources.rest_api.notion]
api_key = "notion api key"

[destination.lancedb]
embedding_model_provider = "sentence-transformers"
embedding_model = "all-MiniLM-L6-v2"

[destination.lancedb.credentials]
uri = ".lancedb"
api_key = "api_key"
embedding_model_provider_api_key = "embedding_model_provider_api_key"

Pass them as environment variable

In [ ]:
import os

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "notion api key"

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"
os.environ["DESTINATION__LANCEDB__CREDENTIALS__API_KEY"] = "api_key"
os.environ["DESTINATION__LANCEDB__CREDENTIALS__EMBEDDING_MODEL_PROVIDER_API_KEY"] = "embedding_model_provider_api_key"

We are going to be using option 2. It's not advisable to paste sensitive information like API keys inside the code, so instead we're going to include them inside the secrets tab in the side panel of the notebook. This will allow us to access the secret values from the notebook.

Since we are using the OSS version of LanceDB and OSS embedding models, we only need to specify the API key for Notion.

Note: You will need to copy the notion API key into the secrets tab under the name SOURCES__REST_API__NOTION__API_KEY. Make sure to enable notebook access after pasting the key.

In [20]:
# Importing the env variables
load_dotenv(override=True)



True

### 3. Write the pipeline code

**Note:** We first go over the code step by step before putting it into runnable cells

1. Import necessary modules (run this cell)


In [7]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

2. Configure the dlt rest api source to connect to and extract the relevant data out from the Notion Rest API

    Our notion space has multiple pages and each page ahs multiple paragraphs (called blocks). To extract all this data form the Notion API, we would first need to get a list of all the page_ids (each page has a unique page_id), and then use the page_id to request the contents from the individual pages. Specifically:

    1. We will first request the page_ids from the /serach endpoint
    2. And then using the return page_ids, we will request the contents form the /blocks/{page_id}/children endpoint.

    With this in mind, we can configure the dlt notion rest api source as follows:

    

In [ ]:
RESTAPIConfig = {
    "client":{
        "base_url": "https://api.notion.com/v1/",
        "auth": {
            "token": dlt.secrets["sources.rest_api.notion.api_key"]
        },
        "headers": {
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
        }
    },
    "resources": [
        {
            "name": "search",
            "endpoint": {
                "path": "search",
                "method": "POST",
                "paginator": PostBodyPaginator(),
                "json": {
                    "query": "workshop",
                    "sort": {
                        "direction": "ascending",
                        "timestamp": "last_edited_time"
                    }
                },
                "data_selector": "results"
            }
        },
        {
            "name": "page_content",
            "endpoint": {
                "path": "blocks/{page_id}/children",
                "paginator": JSONResponsePaginator(),
                "params": {
                    "page_id": {
                        "type": "resolve",
                        "resource": "search",
                        "field": "id"
                    }
                },
            }
        }
    ]
}

Explanation:

1. client: Here we added our base url, headers, and authentication

2. resources: This is a list of endpoints that we wish to request data from (here: /search and /blocks/{page_id}/children)
3. /search endpoint: The Notion API search endpoint allows us to filter pages based on the title. We can specify which pages we want returned based on the parameter "query". For example, if we'd like to return only those pages which has the word "workshop" in the title, then we would set  "query": "workshop" in the json body.


As a response, it returns only page metadata (like page_id). Example response:

In [ ]:
{
"object": "list",
"results": [
    {
    "object": "page",
    "id": "954b67f9-3f87-41db-8874-23b92bbd31ee",
    "created_time": "2022-07-06T19:30:00.000Z",
    "last_edited_time": "2022-07-06T19:30:00.000Z",
    .
    .
    .
],
"next_cursor": null,
"has_more": false,
"type": "page_or_database",
"page_or_database": {}
}

This is how we would define our endpoint configuration for /search

In [ ]:
{
    "name": "search",
    "endpoint": {
        "path": "search",
        "method": "POST",
        "paginator": PostBodyPaginator(),
        "json": {
            "query": "workshop",
            "sort": {
                "direction": "ascending",
                "timestamp": "last_edited_time"
            }
        },
        "data_selector": "results"
    }
}

* paginator allows us to specify the pagination strategy relevant for the API and the endpoint. (More on this later)
* Since /search is a POST endpoint, we can include the json bodu inside the key kson.
* We don't need the whole JSON response, but only the contents inside the field "results". We filter this out by specifying "data_selector":"results.

4. blocks/{page_id}/children endpoint:

* This is a GET point that reutns a list of block objects (in our case, paragraphs) from a specific page.
* Since it accepts page_id as a parameter, we can pass this inside the key params
* We would like to be able to automatically fethc the page_ids returned from the /search endpoint and pass it as parameter into the endpoint blocks/{page_id}/children. We can do this by linking the two resources as follows:

In [ ]:
{
    "name": "page_content",
    "endpoint": {
        "path": "blocks/{page_id}/children",
        "paginator": JSONResponsePaginator(),
        "params": {
            "page_id": {
                "type": "resolve",
                "resource": "search",
                "field": "id"
            }
        },
    }
}

By specifying "type":"resolve", we are letting dlt know that this parameter needs to be resolved from the parent source "search" using the field "id", which corresponds to the page id in the response of /search

Note on pagination:

Different REST APIs might use different strategies handle paginated responses. dlt has a built in support for most common pagination mechanisms, and these can be explicitly passed inside the configurarion like shown above.

However inmost cases, it won't be necessary to explicitly specify the pagination strategy, since dlt detects it automatically.

In case the sepecific pagination is no wupported by dlt yet, then you can also implement a custom paginator. For example, dlt does not have a buil-in paginator for POST methods, so we write our own paginator. We take the code provided in the docs for it and make small modification based on the notion API documentation

In [22]:
from requests import Request, Response


class PosBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None


    def update_state(self, response: Response) -> None:
        # Assuming the API returns and empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False
        
    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        request.json["start_cursor"] = self.cursor

        

3. Extract relevant content form the response body:

    The response returned form the API is a nested JSON which we need to pre-process before using it anywhere, dlt can unnes json automatically, but since the Notion API is a little tricky it's better to pre-process this frist so we have a more structured DB as a result.

    One ay to do this is to pass the JSON response thorugh a transformation function that will extract only the relevant data form the JSON body (we later add thus as a mapping to the resource):

In [23]:
from datetime import datetime, timezone

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response['paragraph']["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
       "block_id": block_id,
       "block_type": block_type,
       "content": content,
       "last_edited_time": last_edited_time,
       "inserted_at_time": datetime.now(timezone.utc)
   }



This is also where you could implement some sort of chunking strategy, but we will omit this in this example as the Notion text is already pre-chunked into paragraphs. Any data pre-processing can also happen here.

Note: If you want to include the parent page in the returned data, you can do so by including response["parent"]["page_id"]. See 200 response example in the Notion docs.

JSON response before the function:

In [ ]:
{
   "object": "list",
   "results": [
     {
       "object": "block",
       "id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
       "created_time": "2022-03-01T19:05:00.000Z",
       "last_edited_time": "2022-03-01T19:05:00.000Z",
       .
       .
       .
       "type": "paragraph",
       "paragraph": {
         "rich_text": [
           {
             .
             .
             .
             "annotations": {
               .
               .
               .

             },
             "plain_text": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
             "href": "https://en.wikipedia.org/wiki/Lacinato_kale"
           }
         ],
         "color": "default"
       }
     }
   ],
   "next_cursor": null,
   "has_more": false,
   "type": "block",
   "block": {}
}

After passing it through the transformation function:

In [ ]:
{
   "block_id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
   "block_type": "paragraph",
   "content": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
   "last_edited_time": "2022-03-01T19:05:00.000Z",
}

Load the data incrementally

Incremental loading is a very important aspect of building scalable data pipelines. It is the technique of loading only the new or changed data since the last run of the pipeline.

In our case, when we first run the pipeline, all paragraphs from the employee handbook will get loaded as separate rows inside a lancedb table. Now if we change the content of one of the paragraphs and re-run the pipeline to update the table, then without doing incremental loading, one of two things may happen depending on the option we choose:

If we choose the option "replace", then the existing data in lancedb will be dropped, and all of the paragraphs will be reloaded.
If we choose the option "append", then the existing rows would remain and all of the paragraphs would be loaded again as new rows resulting in twice as many rows
To ensure that only the new/changed rows are loaded we would need the following pieces:

A column that can keep track of changes in the row (Example: only load rows where last_edited_time is greater than the current maximum last_edited_time)
A primary_key column that uniquely identify a row, so it's possible to track when the row changes
A strategy to resolve changes in a single row (example: drop current and load the changed row)
This behaviour can be configured easily into a dlt rersource:

Pass the incremental column as a parameter inside the resource

In [ ]:
def rest_api_notion_incremental(
 last_edited_time = dlt.sources.incremental(
        "last_edited_time", 
        initial_value="2024-06-26T08:16:00.000Z",
        primary_key=("block_id")
    )
):

We choose the column last_edited_time since it keeps track of whenever a paragraph changes.

Pass the following arguments inside @dlt.resource to define the strategy for dealing with duplicate rows:

* write_disposition="merge": ensures that any duplicate rows are merged on the primary key
* primary_key="block_id": specifies the primary key that we'd like to merge on. In our case, this is block_id, which is a unique id corresponding to each block (paragraph).
* columns={"last_edited_time":{"dedup_sort":"desc"}}: this specifies the deduplication strategy (how we would like to resolve duplicate rows). Here we chose to keep the row with the largest value of last_edited_time.
Putting it together:

In [ ]:
@dlt.resource(
   name="employee_handbook",
   write_disposition="merge",
   primary_key="block_id",
   columns={"last_edited_time":{"dedup_sort":"desc"}}
)
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental(
       "last_edited_time", 
       initial_value="2024-06-26T08:16:00.000Z",
       primary_key=("block_id")
    )
):
    for block in rest_api_notion_resource.add_map(extract_page_content):   
        if not(len(block["content"])):
            continue
        yield block

Here, rest_api_notion_resoure yields the JSON response from the Notion REST API and extract_page_content is the transformation function that we pass the JSON response through.

### 5. Create the pipeline and run it

With our source configured, we can now define the pipeline and run it.

Normally, to do this we would run

In [ ]:
pipeline.run(
 rest_api_notion_incremental,
 table_name="employee_handbook",
 write_disposition="merge"
)

and this would load the data into lancedb normally, without creating any embeddings.

However, we can have lancedb automatically create embeddings and load it along with the normal data using dlt's native adapter for lancedb: lancedb_adapter. It will use the embedding model that we specified in the credentials.

In [ ]:
pipeline.run(
 lancedb_adapter(
   rest_api_notion_incremental,
   embed="content" # The column that we'd like to embed
 )
 table_name="employee_handbook",
 write_disposition="merge"
)

## Run the pipeline

In [8]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "Homework: Employee handbook",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="homework_employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in 1.71 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x0000022B114D1B70> location to store data
Load package 1721688414.8159938 is LOADED and contains no failed jobs


## Visualize the output

In [13]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

df = dbtable.to_pandas()

In [16]:
df.shape

(17, 9)

In [17]:
df

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.02426561, 0.047460843, -0.011796447, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 22:46:56.115702+00:00,1721688414.8159938,lHzqppPI6h8o5Q
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.04966159, 0.10853516, -0.009762599, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 22:46:56.117701+00:00,1721688414.8159938,TyyBJdqQeH1oDg
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316318, 0.17331503, 0.025351722, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.117701+00:00,1721688414.8159938,rdxWxBkCPFSuuw
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974314, 0.105860785, 0.0032906178, -0.02...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.117701+00:00,1721688414.8159938,p01ya+QmSfjYnw
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423313, -0.06457593, 0.065862976, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.118701+00:00,1721688414.8159938,h76zOAdnOpD8Bw
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.00052335917, -0.054883417, 0.043573372, -0....",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.118701+00:00,1721688414.8159938,DpnGp1hJnnEK4w
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.038026277, -0.021509644, 0.04752782, 0.0647...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.118701+00:00,1721688414.8159938,y9hFIVKsAPnR+g
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.058588095, -0.075404435, 0.03377518, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.118701+00:00,1721688414.8159938,VAn+vqbB0hgdLw
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.013599242, 0.0047530443, 0.024835145, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 22:46:56.118701+00:00,1721688414.8159938,r3JLk1B9PnpZEg
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.03206088, 0.024244627, 0.008471343, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 22:46:56.119700+00:00,1721688414.8159938,GvRCdbpZeDKM/w


In [31]:
df.last_edited_time.max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

Now we make change to one of the paragraphs and run the pipeline again to see the effect of incremental loading. We observe two things:

1. The column inserted_at_time only changed for the updated row, implying that only this row was added
2. Looking at the primary key block_id we see that the original row was dropped and the updated row was inserted

# Part 2: Create a RAG bot using Ollama

With the contents from the employee handbook vectorized and stored in LanceDB, we're now ready to create our RAG with Ollama.

What is RAG?

Retrieval Automated Generation (RAG) is the framework of retrieving relevant documents from a database and passing it along with a query into an LLM so that the LLM can generate context-aware responses.

In our case, if we were to ask an LLM questions about our specific employee policies, then we would not get useful responses because the LLM has never seen these policies. A solution to this could be to paste all of the policies into the prompt and then ask our questions. However, this would not be feasible given the limitations on the size of the context window.

We can bypass this limitation using RAG:

Given a user question, we would first embed this question into a vector
Then we would do a vector search on our LanceDB table and retrieve top k results - which would be the most relevant paragraphs corresponding to the question
Finally, we would pass the original question along with the retrieved paragraphs as a prompt into the LLM

1. Install Ollama into the notebook's local runtime

2. Start Ollama using ollama serve. This needs to run in the backgound - so we run it using nohup (to see the output log, open nohup.out).

In [18]:
%%cmd
nohup ollama serve > nohup.out 2>&1 &


Microsoft Windows [Versi�n 10.0.19045.4651]
(c) Microsoft Corporation. Todos los derechos reservados.

(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>nohup ollama serve > nohup.out 2>&1 &

(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>

3. Pull the desired model. We're going to be using llama1-uncensored (takes about 1m to download)

In [19]:
%%cmd
ollama pull llama2-uncensored

Microsoft Windows [Versi�n 10.0.19045.4651]
(c) Microsoft Corporation. Todos los derechos reservados.

(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>ollama pull llama2-uncensored


pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest 
pulling 6aa74acf170f... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 28577ba2177f... 100% ▕████████████████▏   55 B                         
pulling 0025f348941e... 100% ▕████████████████▏   39 B                         
pulling c67e365e770d... 100% ▕████████████████▏  529 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 



(llm-zoomcampt-3.10) c:\Users\User\Documents\llm-zoomcamp-dbeta95\homeworks>

In this next part we're going to be writing functions that accept user question, retrieve the relevant paragraphs from lancedb, and the pass the question and the retrieved pages as input into the ollama chat assistant

In [28]:
!pip install ollama


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


5. Write a function that can retrieve content from lancedb relevant to the user query
  
  With LanceDB, you don't have to explicity embed the question. LanceDB stores information on the embedding model used and automatically embeds the question.

  We use the `db_table.search()` function to query the DB and then limit it to the top 2 most similar results and return that as the context to pass to the RAG.

  Limiting results is important because otherwise there might be too much confusing information. Similarly only picking the top choice might not give enough information.

In [20]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

6. Finally we define a very basic RAG. We define a simple system prompt, retrieve the relevant context for the user query with the function defined above and then send the user question and the context to the `llama2-uncensored` model.

In [23]:
import ollama
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

And we run the RAG! Some example questions you can ask:

* How many vacation days do I get?
* Can I get maternity leave?

**Note**: This is a very basic implementation of a RAG, since this workshop is mainly about data ingestion. So expect some weird answers. If you do stop and restart the cell, you will need to rerun the cell containing `ollama serve` first.

In [24]:
main()

Assistant: Hi there! I hope you are doing well today. Let me help you with this question. According to the information provided, employees receive 30 days of Paid Time Off (PTO) per year. Based on that context, your answer is thirty days. If the employee wishes to check the company's policy about taking more than 30 days of PTO in a year, they should contact their HR department or read through the employee handbook for further information.


KeyboardInterrupt: 